# Read NCOM OoT Gulf of Mexico data
using Zarr/ReferenceFileSystem

In [ ]:
import fsspec
import xarray as xr
import json
import intake
import numpy as np
import hvplot.xarray

#### Load from jsons in zip into a single dataset

In [ ]:
%%time
jsons = fsspec.open_files("zip://*::s3://coawst-public/rsignell/testing/gom1km.zip",
                          s3={"requester_pays": True})

ds_list=[]
for j in jsons:
    with j as afile:
        m = fsspec.get_mapper("reference://", fo=json.load(afile), remote_protocol="s3", 
                              remote_options={"requester_pays": 'true'})
        ds_list.append(xr.open_dataset(m, engine="zarr", drop_variables='reference_time'))

ds = xr.concat(ds_list, dim='time', 
               coords='minimal', data_vars='minimal', compat='override')

In [ ]:
ds.water_u[0,:,:].mean().values

In [ ]:
ds

Drop the singleton depth dimension

In [ ]:
ds = ds.squeeze('depth', drop=True)

Use numpy expression for sqrt -- don't need xarray ufuncs

In [ ]:
ds.water_u

In [ ]:
ds['speed'] = np.sqrt(ds.water_u**2 + ds.water_v**2)

holoviz like lon to be [-180, 180]

In [ ]:
ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180))

Visualize with holoviz, zoom/pan with tools on right:

In [ ]:
ds.speed.hvplot.quadmesh(x='lon', y='lat', rasterize=True, cmap='turbo', geo=True, tiles='OSM')

In [ ]:
ds

In [ ]:
ds.speed[:,700,800].hvplot(grid=True)